# PLEASE USE `Cell => Run All` ON THIS NOTEBOOK (MENU OPTION ABOVE).

# THIS NOTEBOOK WILL TAKE 20-30 MINUTES TO COMPLETE SUCCESSFULLY.

# PLEASE BE PATIENT

# 1. Install the CLI tools

_Note:  Make sure the SageMaker role has access to create EKS Cluster._

In [ ]:
!wget https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_$(uname -s)_amd64.tar.gz

!tar -xzvf eksctl_$(uname -s)_amd64.tar.gz -C /tmp

!mv /tmp/eksctl /usr/local/bin

!eksctl version

In [ ]:
!wget https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/kubectl

!chmod +x ./kubectl

!mv ./kubectl /usr/local/bin

!kubectl version --short --client

In [ ]:
!wget aws-iam-authenticator https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/aws-iam-authenticator

!chmod +x ./aws-iam-authenticator

!mv aws-iam-authenticator /usr/local/bin

!aws-iam-authenticator version

In [ ]:
!wget -O jq https://github.com/stedolan/jq/releases/download/jq-1.6/jq-linux64

!chmod +x ./jq

!cp jq /usr/local/bin

Ignore ERROR and WARNING below

In [ ]:
# !pip install awscli -q --upgrade --user

_Ignore ERROR and WARNING ^^ above ^^_

# 2. Configure the Environment Variables

In [ ]:
%%bash

. ~/.bash_profile

export AWS_REGION=us-east-1 #$(aws configure get region)
echo "export AWS_REGION=${AWS_REGION}" | tee -a ~/.bash_profile

export AWS_CLUSTER_NAME=cluster
echo "export AWS_CLUSTER_NAME=${AWS_CLUSTER_NAME}" | tee -a ~/.bash_profile

echo "Completed"

# 3. Create the EKS Cluster

# THIS WILL TAKE 20-30 MINUTES.  PLEASE BE PATIENT!

# _If you see `retryable error`s below, this is OK!_

In [ ]:
%%bash

###############################
#
# ONLY RUN THIS CELL ONE TIME!!
#
###############################

. ~/.bash_profile

export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS."
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  ACTIVE"
        exit
    fi
else
    # THIS WILL TAKE 20-30 MINUTES.  PLEASE BE PATIENT. 
    eksctl create cluster \
      --name ${AWS_CLUSTER_NAME} \
      --version 1.17 \
      --nodes 4 \
      --node-type c5.xlarge \
      --node-volume-size 100 \
      --region=${AWS_REGION} \
      --zones=${AWS_REGION}a,${AWS_REGION}b \
      --alb-ingress-access
      
    export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)
    echo "export AWS_CLUSTER_STATUS=${AWS_CLUSTER_STATUS}" | tee -a ~/.bash_profile
    echo "Cluster status:  $AWS_CLUSTER_STATUS"
fi

# _Your output will look like this..._

![](img/verify_eks.png)


# _If you see `retryable error`s  ^^ above ^^, this is OK!_

# AFTER 20-30 MINUTES, YOUR CLUSTER WILL BE CREATED.

In [ ]:
%%bash

. ~/.bash_profile

export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please continue."
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  ACTIVE"
        exit
    fi
fi

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}